# Ollama test

In [16]:
import cv2
import ollama
from PIL import Image
import io

# ================= 配置区域 =================
MODEL_NAME = "qwen2.5vl:7b"  

# VIDEO_PATH = "./test_videos/video_24.mp4" 
VIDEO_PATH = "./output_L2CS/video_24.mp4" 
# VIDEO_PATH = "./output_gazelle/video_105.mp4" 

# 如果视频长 10 秒，SAMPLES=10 代表每秒截一张
TARGET_FRAME_COUNT = 20 
# ===========================================

def extract_frames(video_path, target_count=TARGET_FRAME_COUNT):
    """
    从视频中均匀抽取指定数量的帧，并转换为字节流
    """
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"❌ 无法打开视频: {video_path}")
        return []

    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    duration = total_frames / fps if fps > 0 else 0
    
    print(f"🎬 视频信息: 时长 {duration:.2f}s | 总帧数 {total_frames} | FPS {fps}")
    
    # 计算抽帧间隔
    step = max(1, total_frames // target_count)
    
    frame_images = [] 
    
    count = 0
    saved_count = 0
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
            
        # 按照步长抽帧
        if count % step == 0 and saved_count < target_count:
            # OpenCV 默认是 BGR，需要转为 RGB
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            pil_img = Image.fromarray(frame_rgb)
            
            # 为了加快速度和节省显存，可以将图片缩小，例如最大边长 720 或 512
            pil_img.thumbnail((960, 540)) 
            
            # 转为字节流
            img_byte_arr = io.BytesIO()
            pil_img.save(img_byte_arr, format='JPEG', quality=90)
            image_bytes = img_byte_arr.getvalue()
            
            frame_images.append(image_bytes)
            saved_count += 1
            
        count += 1

    cap.release()
    print(f"抽取 {len(frame_images)} 帧用于分析")
    return frame_images

def analyze_video(video_path):
    frames = extract_frames(video_path, target_count=TARGET_FRAME_COUNT)
    
    if not frames:
        return

    prompt = (
        f"You are an expert in micro-expression analysis. \n"
        # "You are analyzing a sequence of video frames marked with RED ARROWS indicating the gaze direction of the woman.\n\n"

        "Your task is to interpret a person’s body language."
        "You will be given a set of video frames representing a clip with one main speaker."
        "Follow the Task Guidelines."
        "Thinking out loud: <your thoughts about which interpretation is better>"

        "### STRICT ANALYSIS PIPELINE\n"
        "Follow these steps to decode the mental state:\n\n"

        "### Step 0: Fact\n"
        "- Given the fact: [She is smiling slightly.]\n"

        "1. **Vector Trajectory Tracing (The 'What')**:\n"
        "   - Compare the red arrow direction changes across timestamps.\n"
        "   - Is the arrow pointing at a fixed target ? OR is it pointing at different locations in each frame (e.g., Down -> Left -> Down-Right -> Straight)?\n"
        "   - **Key Question**: Define the 'Gaze Pattern'. Is it 'Stable/Focused' or 'Wandering/Unfocused'?\n\n"
        
        "2. **Incongruence Check (The 'Why')**:\n"
        "   - Observe her [facial expression] (Mouth/Cheeks). \n"
        "   - Contrast this with the 'Gaze Pattern' identified in Step 1. \n"
        "Is the expression consistent with the gaze behavior? or conflict?\n\n"
        
        "3. **Psychological Inference (The 'Meaning')**:\n"
        "   - Based on the specific [facial expression] + [Gaze Pattern] , what is her emotion?\n\n"
        
        "### Step 4: Final Inference\n"
        "- Based on the steps above, determine the mental state.\n"
        "- Choose between:\n"
        "   A. [Based on these observations, the speaker is likely not feeling joy.] \n"
        "   B. [Based on these observations, the speaker is likely feeling joy.] \n\n"
        
        "Output your reasoning for each step, and then provide the final conclusion."
    )

    # prompt = (     
    #     f"You are a socially intelligent body language expert. Your task is to interpret a person’s body language. "
    #     f"You will be given a set of video frames representing a clip with one main speaker. " 
    #     f"Follow the Task Guidelines and the Response Format.\n"            
    #     f"- You are given one fact about the speaker’s body language, and two possible interpretations.\n"
    #     f"- Think out loud about which interpretation is better given what you see in the images (2-3 sentences).\n"
    #     # add gaze
    #     f"- Please carefully observe the trajectory of the speaker's gaze (indicated by the red arrows) and eye blink in the whole video and think about it\n\n"
    #     f"- Finally, give your answer according to the Response Format.\n\n"
    #     f"Thinking out loud: <your thoughts about which interpretation is better (2-3 sentences)> \n"
    #     f"Answer: <A OR B>\n\n"
    #     f"### Fact: [She is smiling slightly.] \n\n"
    #     f"### Interpretations\n"
    #     f"A. [Based on these observations, the speaker is likely not feeling joy.] \n"
    #     f"B. [Based on these observations, the speaker is likely feeling joy.] \n\n"
    #     f"Please analyze the possible problems and give suggestions for improvement and rehabilitation actions."
    # )

    # prompt = ("describe what you have seen in the images in detail.")

    print("推理中")
    
    # 3. 发送给 Ollama:'images' 参数接受一个列表，我们把所有抽出来的帧都放进去
    try:
        response = ollama.chat(
            model=MODEL_NAME,
            messages=[{
                'role': 'user',
                'content': prompt,
                'images': frames  # 这里传入所有帧
            }],
                options={
                    'temperature': 0.1,
                    'num_ctx': 20000
                }
        )
        
        print("\n" + "="*20 + " 识别结果 " + "="*20)
        print(response['message']['content'])
        print("="*50)
        
    except Exception as e:
        print(f"❌ 推理出错: {e}")


analyze_video(VIDEO_PATH)

🎬 视频信息: 时长 3.52s | 总帧数 81 | FPS 23
抽取 20 帧用于分析
推理中

==================== 识别结果 ====================
### Step 0: Fact
- Given the fact: [She is smiling slightly.]

### Step 1: Vector Trajectory Tracing (The 'What')
- **Red Arrow Direction**: The red arrow in the frames points to the woman's face, indicating that she is the focus of attention.
- **Gaze Pattern**: The arrow remains consistently pointing to her face, suggesting a stable and focused gaze.

### Step 2: Incongruence Check (The 'Why')
- **Facial Expression**: The woman is smiling slightly. This is a positive expression, typically associated with happiness or contentment.
- **Incongruence**: The gaze pattern is stable and focused on her face, and the facial expression is consistent with a positive emotion. There is no incongruence between the gaze and the facial expression.

### Step 3: Psychological Inference (The 'Meaning')
- Based on the stable gaze and the positive facial expression, the woman appears to be feeling joy or co

# Ollama all dataset

In [ ]:
import json
import os
import cv2
import io
import ollama
from PIL import Image

# ================= 配置区域 =================
# 你的 Ollama 模型名字
MODEL_NAME = "qwen2.5vl:7b"  # 确保你 ollama pull 了这个模型

# 视频文件夹的路径
VIDEO_FOLDER = "./videos"
INPUT_FILE = "vibe_task.jsonl"
OUTPUT_FILE = "results.jsonl"

# 抽帧数量：根据显存大小调整。
# 7B 模型在 Mac 上建议 8-16 帧，帧数越多越卡，但细节越好。
TARGET_FRAME_COUNT = 8
# ===========================================

def extract_frames(video_path, target_count=TARGET_FRAME_COUNT):
    """
    读取视频文件，均匀抽取指定数量的帧，并转为字节流列表
    """
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"❌ 无法打开视频: {video_path}")
        return []

    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    # 简单的均匀抽帧逻辑
    step = max(1, total_frames // target_count)
    frame_images = []
    
    count = 0
    saved_count = 0
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
            
        if count % step == 0 and saved_count < target_count:
            # OpenCV BGR -> RGB
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            pil_img = Image.fromarray(frame_rgb)
            
            # 缩放图片以节省显存 (可选，建议长边不超过 768 或 512)
            # Qwen2.5-VL 支持任意分辨率，但太大会导致推理慢
            pil_img.thumbnail((512, 512)) 
            
            # 转为字节流
            img_byte_arr = io.BytesIO()
            pil_img.save(img_byte_arr, format='JPEG', quality=70)
            frame_images.append(img_byte_arr.getvalue())
            saved_count += 1
            
        count += 1

    cap.release()
    return frame_images

# def process_data():
    # 检查输出文件是否存在，如果存在，统计行数以便断点续传（可选）
processed_ids = set()
if os.path.exists(OUTPUT_FILE):
    with open(OUTPUT_FILE, 'r', encoding='utf-8') as f:
        for line in f:
            if line.strip():
                try:
                    data = json.loads(line)
                    processed_ids.add(data.get("task_id"))
                except:
                    pass

with open(INPUT_FILE, 'r', encoding='utf-8') as f_in, \
        open(OUTPUT_FILE, 'a', encoding='utf-8') as f_out: # 使用 'a' 模式追加
    
    for line_number, line in enumerate(f_in):
        if not line.strip(): continue
        
        item = json.loads(line)
        task_id = item.get("task_id")
        
        # 简单的断点续传逻辑
        if task_id in processed_ids:
            print(f"Skipping processed task: {task_id}")
            continue

        video_filename = item.get("video")
        fact_text = item.get("fact")
        inference_A = item.get("inference_A")
        inference_B = item.get("inference_B")
        
        # --- 步骤 1: 获取视频绝对路径 ---
        abs_video_path = os.path.abspath(os.path.join(VIDEO_FOLDER, video_filename))
        if not os.path.exists(abs_video_path):
            print(f"Error: Video file not found: {abs_video_path}")
            continue
        
        # --- 步骤 2: (核心修改) 抽取视频帧 ---
        # 本地 Ollama 不支持直接传文件路径，必须传图片列表
        print(f"Processing Task: {task_id} | Extracting frames from {video_filename}...")
        frames = extract_frames(abs_video_path, target_count=TARGET_FRAME_COUNT)
        
        if not frames:
            print(f"Error: No frames extracted for {task_id}")
            continue

        # --- 步骤 3: 构建 Prompt ---
        text_prompt = (     
            f"You are a socially intelligent body language expert. Your task is to interpret a person’s body language. "
            f"You will be given a set of video frames representing a clip with one main speaker. " 
            f"Follow the Task Guidelines and the Response Format.\n"            
            f"- You are given one fact about the speaker’s body language, and two possible interpretations.\n"
            f"- Think out loud about which interpretation is better given what you see in the images (2-3 sentences).\n"
            f"- Finally, give your answer according to the Response Format.\n\n"
            f"Thinking out loud: <your thoughts about which interpretation is better (2-3 sentences)> \n"
            f"Answer: <A OR B>\n\n"
            f"### Fact\n{fact_text}\n\n"
            f"### Interpretations\n"
            f"A. {inference_A}\n"
            f"B. {inference_B}\n\n"
            f"Please analyze the possible problems and give suggestions for improvement and rehabilitation actions."
        )

        try:
            # --- 步骤 4: (核心修改) 调用 Ollama ---
            response = ollama.chat(
                model=MODEL_NAME,
                messages=[{
                    'role': 'user',
                    'content': text_prompt,
                    'images': frames  # 关键：传入抽取的帧数据列表
                }],
                options={
                    'temperature': 0.1,
                    # "num_batch": 256,
                    # 'num_ctx': 8192
                    # "num_gpu": 20
                }
            )
            
            response_content = response['message']['content']
            print(f"✅ Result for {task_id} generated.")

            # 保存结果
            result = {
                "task_id": task_id,
                "model_output": response_content,
                "ground_truth": item.get("answer"),
                "inference_A": inference_A, 
                "inference_B": inference_B
            }
            
            f_out.write(json.dumps(result, ensure_ascii=False) + "\n")
            f_out.flush()
            
        except Exception as e:
            print(f"❌ Error processing {task_id}: {e}")

Skipping processed task: video_1
Skipping processed task: video_1
Skipping processed task: video_2
Skipping processed task: video_2
Skipping processed task: video_2
Skipping processed task: video_2
Processing Task: video_3 | Extracting frames from video_3.mp4...
✅ Result for video_3 generated.
Processing Task: video_3 | Extracting frames from video_3.mp4...
✅ Result for video_3 generated.
Processing Task: video_3 | Extracting frames from video_3.mp4...
✅ Result for video_3 generated.
Processing Task: video_4 | Extracting frames from video_4.mp4...
✅ Result for video_4 generated.
Processing Task: video_5 | Extracting frames from video_5.mp4...
✅ Result for video_5 generated.
Processing Task: video_5 | Extracting frames from video_5.mp4...
✅ Result for video_5 generated.
Processing Task: video_5 | Extracting frames from video_5.mp4...
✅ Result for video_5 generated.
Processing Task: video_5 | Extracting frames from video_5.mp4...
✅ Result for video_5 generated.
Processing Task: video_5 |

# output process

In [2]:
import json
import re
import pandas as pd

# =================配置区域=================
INPUT_FILE = "results.jsonl"  # 你的结果文件路径
# =========================================

def extract_model_answer(text):
    """
    从模型的输出中提取 A 或 B。
    策略：找 "Answer:" 后面的第一个 A 或 B，或者找最后出现的 A/B。
    """
    if not isinstance(text, str):
        return "Unknown"
    
    # 优先匹配明确的格式 "Answer: A" 或 "Answer: <A>"
    match = re.search(r"Answer:\s*[:<]?\s*([AB])\b", text, re.IGNORECASE)
    if match:
        return match.group(1).upper()
    
    # 如果没找到标准格式，尝试找文本最后出现的选项（这是一个兜底策略）
    # 也就是寻找最后出现的 "A" 或 "B" 字符
    matches = re.findall(r"\b([AB])\b", text)
    if matches:
        return matches[-1].upper()
    
    return "Unknown"

def detect_question_type(item):
    """
    根据选项文本内容判断是 NT 还是 NF。
    逻辑依据：
    - NT (Negation True): 正确选项包含否定词。
    - NF (Negation False): 错误选项包含否定词。
    """
    ground_truth = item.get("ground_truth", "").strip().upper()
    inference_A = item.get("inference_A", "")
    inference_B = item.get("inference_B", "")
    
    if ground_truth not in ["A", "B"]:
        return "Error"

    # 1. 确定哪个文本是“正确选项”，哪个是“错误选项”
    if ground_truth == "A":
        correct_text = inference_A
        incorrect_text = inference_B
    else:
        correct_text = inference_B
        incorrect_text = inference_A
        
    # 2. 定义否定词列表 (根据 VIBE 论文逻辑)
    negation_keywords = ["not", "don't", "doesn't", "didn't", "cannot", "never"]
    
    def has_negation(text):
        text_lower = text.lower()
        return any(kw in text_lower for kw in negation_keywords)

    is_correct_negated = has_negation(correct_text)
    is_incorrect_negated = has_negation(incorrect_text)

    # 3. 分类逻辑
    if is_correct_negated:
        return "NT"  # 正确答案是否定的 -> Negation True
    elif is_incorrect_negated:
        return "NF"  # 错误答案是否定的 -> Negation False
    else:
        return "Standard" # 其他类型（论文中的 Fixed/Distractor 若无否定词通常落入此类）

def evaluate_vibe(file_path):
    data = []
    
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            if not line.strip(): continue
            item = json.loads(line)
            
            # 1. 提取预测结果
            prediction = extract_model_answer(item.get("model_output"))
            truth = item.get("ground_truth")
            
            # 2. 判断正误
            is_correct = (prediction == truth)
            
            # 3. 判断题目类型
            q_type = detect_question_type(item)
            
            data.append({
                "Task ID": item.get("task_id"),
                "Type": q_type,
                "Correct": is_correct,
                "Model Output": prediction,
                "Ground Truth": truth
            })
            
    df = pd.DataFrame(data)
    
    # 计算各类别准确率
    results = {}
    
    # 计算 Overall (All)
    results['All'] = df['Correct'].mean() * 100
    
    # 计算各子类别 (NT, NF, Standard)
    for q_type in ['NT', 'NF', 'Standard']:
        subset = df[df['Type'] == q_type]
        if len(subset) > 0:
            results[q_type] = subset['Correct'].mean() * 100
        else:
            results[q_type] = 0.0
            
    # 格式化输出表格
    print("\n" + "="*50)
    print("VIBE Evaluation Results (Accuracy %)")
    print("="*50)
    
    # 创建类似论文的表格 DataFrame
    # 注意：这里用 'Standard' 代替了 Fixed/Distractor，因为那是内部元数据
    # 如果你想区分 Fixed/Distractor，需要在 jsonl 里有额外的 tag 字段
    
    summary_df = pd.DataFrame([{
        "Model": "Qwen2.5-VL-7B",
        "Standard": f"{results['Standard']:.1f}",
        "NF": f"{results['NF']:.1f}",
        "NT": f"{results['NT']:.1f}",
        "All": f"{results['All']:.1f}"
    }])
    # summary_df = pd.DataFrame([{
    #         "Model": "Qwen2.5-VL-3B",
    #         "Standard": f"70.1",
    #         "NF": f"81.1",
    #         "NT": f"30.6",
    #         "All": f"68.6"
    #     }])
    
    print(summary_df.to_string(index=False))
    print("="*50)
    
    # 打印详细统计
    print(f"\n样本统计:\nTotal: {len(df)}\nNT count: {len(df[df['Type']=='NT'])}\nNF count: {len(df[df['Type']=='NF'])}\nStandard count: {len(df[df['Type']=='Standard'])}")

    return df

if __name__ == "__main__":
    # 请确保你的 results.jsonl 文件里包含了 inference_A 和 inference_B
    # 如果没有，你需要先用代码把原始数据 merge 进去
    if "inference_A" not in open(INPUT_FILE).readline():
        print("错误：JSONL 文件中缺少 'inference_A/B' 字段。无法进行 NT/NF 分类。")
        print("请修改之前的脚本，在写入 results.jsonl 时把原始 item 的所有字段都写进去。")
    else:
        df = evaluate_vibe(INPUT_FILE)


VIBE Evaluation Results (Accuracy %)
        Model Standard   NF   NT  All
Qwen2.5-VL-7B     74.2 88.9 44.9 75.6

样本统计:
Total: 991
NT count: 138
NF count: 369
Standard count: 484
